# Module 3. Deployment on MMS(Multi Model Server)
---

본 모듈에서는 모델의 배포(deployment)를 수행합니다. 노트북 실행에는 약 15분 가량 소요되며, 핸즈온 실습 시에는 25분을 권장드립니다.

<br>

## 1. Inference script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 추론 스크립트인 `inference.py`를 저장합니다.<br>

이 스크립트는 SageMaker 상에서 MMS(Multi Model Server)를 쉽고 편하게 배포할 수 이는 high-level 툴킷인 SageMaker inference toolkit의 인터페이스를
사용하고 있으며, 여러분께서는 인터페이스에 정의된 핸들러(handler) 함수들만 구현하시면 됩니다.

#### MMS(Multi Model Server)란?
- [https://github.com/awslabs/multi-model-server](https://github.com/awslabs/multi-model-server) (2017년 12월 초 MXNet 1.0 릴리스 시 최초 공개, MXNet용 모델 서버로 시작)
- Prerequisites: Java 8, MXNet (단, MXNet 사용 시에만)
- MMS는 프레임워크에 구애받지 않도록 설계되었기 때문에, 모든 프레임워크의 백엔드 엔진 역할을 할 수 있는 충분한 유연성을 제공합니다.
- SageMaker MXNet 추론 컨테이너는 SageMaker inference toolkit으로 MMS를 래핑하여 사용합니다.

In [1]:
%%writefile ./src/inference.py

import os
import pandas as pd
import gluonts 
import numpy as np
import argparse
import json
import pathlib
from mxnet import gpu, cpu
from mxnet.context import num_gpus
import matplotlib.pyplot as plt

from gluonts.dataset.util import to_pandas
from gluonts.mx.distribution import DistributionOutput, StudentTOutput, NegativeBinomialOutput, GaussianOutput
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions, backtest_metrics
from gluonts.model.predictor import Predictor
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset


def model_fn(model_dir):
    path = pathlib.Path(model_dir)   
    predictor = Predictor.deserialize(path)
    print("model was loaded successfully")
    return predictor


def transform_fn(model, request_body, content_type='application/json', accept_type='application/json'):

    data = json.loads(request_body)
    target_test_df = pd.DataFrame(data['value'], index=data['timestamp'])
    target = target_test_df.values
    num_series = target_test_df.shape[1]
    start_dt = target_test_df.index[0]
    test_lst = []
    
    for i in range(0, num_series):
        target_vec = target[:, i]
        dic = {FieldName.TARGET: target_vec, 
               FieldName.START: start_dt} 
        test_lst.append(dic)
        
    test_ds = ListDataset(test_lst, freq='1D')
    
    response_body = {}
    forecast_it = model.predict(test_ds)
    for idx, f in enumerate(forecast_it):
        response_body[f'item_{idx}'] = f.samples.mean(axis=0).tolist()

    return json.dumps(response_body)

Overwriting ./src/inference.py


<br>

## 2. Test Inference code 
---

엔드포인트 배포 전, 추론 스크립트를 검증합니다. 

In [2]:
%store -r

In [3]:
from src.inference import model_fn, transform_fn
import json
import numpy as np
import pandas as pd

# Prepare test data
target_test_df = pd.read_csv("data/target_train.csv")
target_test_df.set_index(target_test_df.columns[0], inplace=True)
num_series = 2
pred_length = 21
target_test_df = target_test_df.iloc[pred_length:,:num_series]
input_data = {'value': target_test_df.values.tolist(), 'timestamp': target_test_df.index.tolist()}
request_body = json.dumps(input_data)

# Test inference script 
model = model_fn('./model')
response = transform_fn(model, request_body)
outputs = json.loads(response)
print(outputs['item_0'])

model was loaded successfully
[13.779692649841309, 16.831052780151367, 17.169921875, 18.444971084594727, 20.433433532714844, 20.80752944946289, 21.781984329223633, 14.678018569946289, 16.983680725097656, 16.961782455444336, 18.321735382080078, 19.852739334106445, 20.979887008666992, 21.757274627685547, 14.305449485778809, 16.792940139770508, 17.346654891967773, 18.061120986938477, 20.07729721069336, 20.284381866455078, 20.880613327026367]


<br>

## 3. Local Endpoint Inference
---

충분한 검증 및 테스트 없이 훈련된 모델을 곧바로 실제 운영 환경에 배포하기에는 많은 위험 요소들이 있습니다. 따라서, 로컬 모드를 사용하여 실제 운영 환경에 배포하기 위한 추론 인스턴스를 시작하기 전에 노트북 인스턴스의 로컬 환경에서 모델을 배포하는 것을 권장합니다. 이를 로컬 모드 엔드포인트(Local Mode Endpoint)라고 합니다.

In [4]:
import os
import time
import sagemaker
from sagemaker.mxnet import MXNetModel
role = sagemaker.get_execution_role()

In [5]:
local_model_path = f'file://{os.getcwd()}/model/model.tar.gz'
endpoint_name = "local-endpoint-store-item-demand-forecast-{}".format(int(time.time()))


아래 코드 셀을 실행 후, 로그를 확인해 보세요. MMS에 대한 세팅값들을 확인하실 수 있습니다.

```bash
algo-1-u3xwd_1  | MMS Home: /usr/local/lib/python3.6/site-packages
algo-1-u3xwd_1  | Current directory: /
algo-1-u3xwd_1  | Temp directory: /home/model-server/tmp
algo-1-u3xwd_1  | Number of GPUs: 0
algo-1-u3xwd_1  | Number of CPUs: 2
algo-1-u3xwd_1  | Max heap size: 878 M
algo-1-u3xwd_1  | Python executable: /usr/local/bin/python3.6
algo-1-u3xwd_1  | Config file: /etc/sagemaker-mms.properties
algo-1-u3xwd_1  | Inference address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Management address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Model Store: /.sagemaker/mms/models
...
```

In [6]:
local_model = MXNetModel(model_data=local_model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.6.0',
                         py_version='py3')

predictor = local_model.deploy(instance_type='local', 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True)

Attaching to gd6rep7zhp-algo-1-vi9h3
gd6rep7zhp-algo-1-vi9h3 | Collecting pandas==1.1.5
gd6rep7zhp-algo-1-vi9h3 |   Downloading pandas-1.1.5-cp36-cp36m-manylinux1_x86_64.whl (9.5 MB)
     |████████████████████████████████| 9.5 MB 20.5 MB/s eta 0:00:01
gd6rep7zhp-algo-1-vi9h3 | Collecting gluonts==0.6.7
gd6rep7zhp-algo-1-vi9h3 |   Downloading gluonts-0.6.7-py3-none-any.whl (569 kB)
     |████████████████████████████████| 569 kB 57.9 MB/s eta 0:00:01
gd6rep7zhp-algo-1-vi9h3 | Collecting toolz~=0.10
gd6rep7zhp-algo-1-vi9h3 |   Downloading toolz-0.11.1-py3-none-any.whl (55 kB)
     |████████████████████████████████| 55 kB 6.8 MB/s  eta 0:00:01
gd6rep7zhp-algo-1-vi9h3 | Collecting ujson~=1.35
gd6rep7zhp-algo-1-vi9h3 |   Downloading ujson-1.35.tar.gz (192 kB)
     |████████████████████████████████| 192 kB 48.3 MB/s eta 0:00:01
gd6rep7zhp-algo-1-vi9h3 | Requirement already satisfied: matplotlib~=3.0 in /usr/local/lib/python3.6/site-packages (from gluonts==0.6.7->-r /opt/ml/model/code/requirem

gd6rep7zhp-algo-1-vi9h3 | 2021-04-07 05:29:00,179 [INFO ] pool-1-thread-10 ACCESS_LOG - /172.18.0.1:47946 "GET /ping HTTP/1.1" 200 26
!

로컬에서 컨테이너를 배포했기 때문에 컨테이너가 현재 실행 중임을 확인할 수 있습니다.

In [7]:
!docker ps

CONTAINER ID        IMAGE                                                                        COMMAND                  CREATED             STATUS              PORTS                              NAMES
1f993fbfe488        763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-cpu-py3   "python /usr/local/b…"   15 seconds ago      Up 13 seconds       0.0.0.0:8080->8080/tcp, 8081/tcp   gd6rep7zhp-algo-1-vi9h3


### Inference using SageMaker SDK

SageMaker SDK의 `predict()` 메서드로 쉽게 추론을 수행할 수 있습니다. 

In [8]:
outputs = predictor.predict(input_data)
print(outputs)

gd6rep7zhp-algo-1-vi9h3 | 2021-04-07 05:29:03,449 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
gd6rep7zhp-algo-1-vi9h3 | 2021-04-07 05:29:03,484 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
gd6rep7zhp-algo-1-vi9h3 | 2021-04-07 05:29:03,518 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
gd6rep7zhp-algo-1-vi9h3 | 2021-04-07 05:29:03,542 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
gd6rep7zhp-algo-1-vi9h3 | 2021-04-07 05:29:03,588 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
gd6rep7zhp-algo-1-vi9h3 | 2021-04-07 05:29:03,594 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle -

### Inference using Boto3 SDK

SageMaker SDK의 `predict()` 메서드로 추론을 수행할 수도 있지만, 이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.<br>
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리
SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다.

참고로 `invoke_endpoint()` 호출을 위한 런타임 클라이언트 인스턴스 생성 시, 로컬 배포 모드에서는 `sagemaker.local.LocalSagemakerRuntimeClient()`를 호출해야 합니다.

In [9]:
client = sagemaker.local.LocalSagemakerClient()
runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = local_model.endpoint_name

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/x-npy',
    Body=json.dumps(input_data)
    )
outputs = response['Body'].read().decode()

gd6rep7zhp-algo-1-vi9h3 | 2021-04-07 05:29:05,144 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 206
gd6rep7zhp-algo-1-vi9h3 | 2021-04-07 05:29:05,145 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:48026 "POST /invocations HTTP/1.1" 200 208


In [10]:
outputs

'{"item_0": [14.173724174499512, 16.966928482055664, 17.466144561767578, 18.601024627685547, 20.091693878173828, 21.141592025756836, 21.955942153930664, 14.829607963562012, 17.287033081054688, 17.25757598876953, 18.471174240112305, 19.975059509277344, 20.952308654785156, 21.624500274658203, 14.430057525634766, 16.90169906616211, 16.915874481201172, 18.294790267944336, 20.201583862304688, 20.36090850830078, 20.75102996826172], "item_1": [36.88957595825195, 45.34449768066406, 46.399715423583984, 48.66847229003906, 52.84553146362305, 56.37529754638672, 59.38008117675781, 39.7441291809082, 46.44867706298828, 46.605247497558594, 48.831932067871094, 53.474056243896484, 55.164127349853516, 58.720394134521484, 38.64341354370117, 45.084049224853516, 45.72357940673828, 48.84552764892578, 52.25189971923828, 55.30583190917969, 57.92888641357422]}'

### Local Mode Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.

In [11]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint_config(EndpointConfigName=endpoint_name)
    model_name = response['ProductionVariants'][0]['ModelName']

    client.delete_model(ModelName=model_name)    
    client.delete_endpoint(EndpointName=endpoint_name)
    client.delete_endpoint_config(EndpointConfigName=endpoint_name)    
    
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint: {endpoint_name}')
    print(f'--- Deleted endpoint_config: {endpoint_name}')    
delete_endpoint(client, endpoint_name)

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: mxnet-inference-2021-04-07-05-28-44-752
--- Deleted endpoint: local-endpoint-store-item-demand-forecast-1617773324
--- Deleted endpoint_config: local-endpoint-store-item-demand-forecast-1617773324


<br>

## 4. SageMaker Hosted Endpoint Inference
---

이제 실제 운영 환경에 엔드포인트 배포를 수행해 보겠습니다. 로컬 모드 엔드포인트와 대부분의 코드가 동일하며, 모델 아티팩트 경로(`model_data`)와 인스턴스 유형(`instance_type`)만 변경해 주시면 됩니다. SageMaker가 관리하는 배포 클러스터를 프로비저닝하는 시간이 소요되기 때문에 추론 서비스를 시작하는 데에는 약 5~10분 정도 소요됩니다.

In [12]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

In [13]:
model_path = os.path.join(s3_model_dir, "model.tar.gz")
endpoint_name = "endpoint-store-item-demand-forecast-{}".format(int(time.time()))

In [14]:
model = MXNetModel(model_data=model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.6.0',
                         py_version='py3')

predictor = model.deploy(instance_type="ml.c5.large", 
                         initial_instance_count=1, 
                         endpoint_name=endpoint_name,
                         wait=True)

-----------!

추론을 수행합니다. 로컬 모드의 코드와 동일합니다.

In [15]:
outputs = predictor.predict(input_data)
print(outputs)

{'item_0': [14.173725128173828, 16.966930389404297, 17.466144561767578, 18.601022720336914, 20.091691970825195, 21.141592025756836, 21.95594024658203, 14.829608917236328, 17.287033081054688, 17.25757598876953, 18.471172332763672, 19.97505760192871, 20.952306747436523, 21.624502182006836, 14.430057525634766, 16.901700973510742, 16.91587257385254, 18.294790267944336, 20.201581954956055, 20.360910415649414, 20.75102996826172], 'item_1': [36.88957977294922, 45.34449768066406, 46.399715423583984, 48.6684684753418, 52.84553146362305, 56.37529754638672, 59.380088806152344, 39.7441291809082, 46.44867706298828, 46.605247497558594, 48.831939697265625, 53.474056243896484, 55.164127349853516, 58.720394134521484, 38.64341354370117, 45.084049224853516, 45.72357940673828, 48.845523834228516, 52.25190353393555, 55.30583572387695, 57.92888259887695]}


In [16]:
import boto3
client = boto3.client('sagemaker')
runtime_client = boto3.client('sagemaker-runtime')
endpoint_name = model.endpoint_name

In [17]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(input_data)
    )
outputs = response['Body'].read().decode()

In [18]:
outputs

'{"item_0": [14.213187217712402, 17.065452575683594, 17.46732521057129, 18.617515563964844, 20.265504837036133, 21.041406631469727, 21.84316635131836, 14.703466415405273, 17.2779483795166, 16.960163116455078, 18.597017288208008, 19.943065643310547, 21.116111755371094, 21.65890121459961, 14.287104606628418, 16.797958374023438, 17.18313217163086, 18.074817657470703, 20.24350929260254, 20.58819580078125, 20.979265213012695], "item_1": [36.503841400146484, 44.464195251464844, 46.19575500488281, 48.420433044433594, 53.023963928222656, 56.451480865478516, 59.3340950012207, 39.41472625732422, 46.63825225830078, 46.637428283691406, 48.9912109375, 53.266822814941406, 54.93186569213867, 58.12149429321289, 37.74271011352539, 44.54322814941406, 44.97233581542969, 48.78057098388672, 51.787471771240234, 54.63759231567383, 57.266326904296875]}'

### SageMaker Hosted Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 불필요한 과금을 피하기 위해 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있으며, UI에서도 쉽게 삭제할 수 있습니다.

In [19]:
delete_endpoint(client, endpoint_name)

--- Deleted model: mxnet-inference-2021-04-07-05-29-16-818
--- Deleted endpoint: endpoint-store-item-demand-forecast-1617773356
--- Deleted endpoint_config: endpoint-store-item-demand-forecast-1617773356
